## Carga de librerias

In [ ]:
import enum
import tqdm
from typing import Generator, Dict, List, Tuple
import pickle
import math
import random
from gensim.parsing.preprocessing import remove_stopword_tokens
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np
from nltk.corpus import stopwords
import spacy
from spacy.lang.en import English
from datetime import datetime
from collections import defaultdict
import pandas as pd

## Definicion de Modelo (clases y funciones)
Definimos las funciones necesarias para nuestro pipeline, y definimos una clase para nuestro modelo de analisis de sentimiento

In [106]:
nlp = spacy.load('en_core_web_trf')
class Sentiments(enum.Enum):
    POS = 'POS'
    NEG = 'NEG'
    
def split_data(data: List, weights: List = (0.7, 0.15, 0.15)):
    split = {
        'train': [],
        'test': [],
        'validation': [],
    }
    for word in data:
        subset = random.choices(['train', 'test', 'validation'], weights=weights)[0]
        split[subset].append(word)

    return split

def sentences_to_words(sentences: List[str]) -> List[List[str]]:
    words = []
    pbar=tqdm.tqdm(range(len(sentences)))
    pbar.set_description('Sentences to Words')
    for i in pbar:    
    #for sentence in sentences:
        # https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess
        words.append(simple_preprocess(str(sentences[i]), deacc=True))  # deacc=True elimina la puntuación
    return words

def remove_stopwords(documents: List[List[str]]) -> List[List[str]]:
    pbar=tqdm.tqdm(range(len(documents)))
    pbar.set_description('Remove StopWords')
    docs=[]
    for i in pbar:  
        docs.append(remove_stopword_tokens(documents[i], stopwords=stopwords.words('english')))
        #docs.append([word for word in simple_preprocess(str(documents[i])) if word not in stopwords.words('english')])
    
    return docs

def create_ngrams(ngram_model_lst, documents: List[List[str]]):
    pbar=tqdm.tqdm(range(len(ngram_model_lst)))
    pbar.set_description('Create N-grams')
    for i in pbar: 
    #for ngram_model in ngram_model_lst:
        documents=[ngram_model_lst[i][doc] for doc in documents]
    return documents

def learn_ngrams(n,min_c,th,documents: List[List[str]]) -> List[List[str]]:
    # We learn bigrams
    #  https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases
    print('learning N-grams')
    ngram_model_lst=[]
    ngram = Phrases(documents,min_count=min_c, threshold=th)
    # we reduce the bigram model to its minimal functionality
    ngram_mod = Phraser(ngram)
    ngram_model_lst.append(ngram_mod)
    if n > 2:
        for i in range(3,n+1):
            documents=[ngram_model_lst[i-3][doc] for doc in documents]
            ngram_model_lst.append(Phraser(Phrases(documents,min_count=min_c, threshold=th)))

    # we apply the bigram model to our documents
    return ngram_model_lst



def lemma(nlp: English, texts: List[List[str]], allowed_postags: List = None) -> List[List[str]]:
    if allowed_postags is None:
        allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV']

    """https://spacy.io/api/annotation"""
    texts_out = []
    pbar=tqdm.tqdm(range(len(texts)))
    pbar.set_description('Lemmatization')
    for i in pbar: 
    #for sent in texts:
        doc = nlp(" ".join(texts[i]),disable=['parser','ner'])
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def tokenize(stopwords,lemmatization,ngrams,documents: List[str], ngram_model_lst) -> List[List[str]]:

    document_words = sentences_to_words(documents)
    if lemmatization:
        document_words = lemma(nlp, document_words)
    if stopwords:
        document_words = remove_stopwords(document_words)
    if ngrams:
        document_words = create_ngrams(ngram_model_lst, document_words)
    return document_words

def train_tokenize(stopwords,lemmatization,documents: List[str]) -> List[List[str]]:

    document_words = sentences_to_words(documents)
    if lemmatization:
        document_words = lemma(nlp, document_words)
    if stopwords:
        document_words = remove_stopwords(document_words)


    return document_words

def create_dictionary(documents: List[List[str]]):
    return Dictionary(documents)



class NB_sentiment_analysis():
    def __init__(self,**kwargs):
        #definicion de Hyperparametros
        self.ngrams = kwargs.get('ngrams', True)
        self.stopwords = kwargs.get('stopwords', True)
        self.lemmatization = kwargs.get('lemmatization', True)        
        self.n = kwargs.get('n', 2)
        self.ngram_mincount = kwargs.get('ngram_mincount', 5)
        self.ngram_th = kwargs.get('ngram_th', 10)        
        #variables auxiliares
        self.ngram_model_lst=[]
        
    def fit(self,train_POS,train_NEG,verbose=False):
        print(datetime.now())

        if self.ngrams:
            self.ngram_model_lst=learn_ngrams(self.n,self.ngram_mincount,self.ngram_th,train_POS+train_NEG)
            print('Negative class N-grams')
            tokenized_NEG = create_ngrams(self.ngram_model_lst, train_NEG)
            print('Positive class N-grams')
            tokenized_POS = create_ngrams(self.ngram_model_lst, train_POS)
        else:
            tokenized_NEG=train_NEG
            tokenized_POS=train_POS
            
        positive_words = [item for sublist in tokenized_POS for item in sublist]
        negative_words = [item for sublist in tokenized_NEG for item in sublist]
        self.dictionary = create_dictionary([negative_words, positive_words])
        positive_bow = self.dictionary.doc2bow(positive_words) 
        negative_bow = self.dictionary.doc2bow(negative_words)
        total_negative_words = len(negative_words) + len(self.dictionary)
        total_positive_words = len(positive_words) + len(self.dictionary)
        self.negative_word_probs = {}
        for id, count in negative_bow:
            self.negative_word_probs[self.dictionary[id]] = {
                'id': id,
                'logprob': np.log((count + 1)/total_negative_words),
                }
        self.negative_word_probs = defaultdict(lambda: {'id': -1,'logprob': np.log(1/total_negative_words)}, self.negative_word_probs)

        self.positive_word_probs = {}
        for id, count in positive_bow:
            self.positive_word_probs[self.dictionary[id]] = {
                'id': id,
                'logprob': np.log((count + 1)/total_positive_words),
            }
        self.positive_word_probs = defaultdict(lambda: {'id': -1,'logprob': np.log(1/total_positive_words)}, self.positive_word_probs)
        
        self.negative_prob = np.log(len(negative_words) / (len(negative_words) + len(positive_words)))
        self.positive_prob = np.log(len(positive_words) / (len(negative_words) + len(positive_words)))
        print(datetime.now())
    
    def predict(self,document):
        doc=[document]
        tokenized_doc=tokenize(self.stopwords,self.lemmatization,self.ngrams,doc, self.ngram_model_lst)
        pos_prob=self.positive_prob
        neg_prob=self.negative_prob
        for token in tokenized_doc[0]:
            pos_prob+=self.positive_word_probs[token]['logprob']
            neg_prob+=self.negative_word_probs[token]['logprob']   
        if pos_prob > neg_prob:
            sentiment=Sentiments.POS
        else:
            sentiment=Sentiments.NEG
        return sentiment
    
    def val_test(self,pos,neg):
        if self.ngrams:
            tokenized_POS=create_ngrams(self.ngram_model_lst, pos)
            tokenized_NEG=create_ngrams(self.ngram_model_lst, neg)
        else:
            tokenized_NEG=neg
            tokenized_POS=pos
        tp=0
        fp=0
        tn=0
        fn=0
        pos_prob=self.positive_prob
        neg_prob=self.negative_prob
        pbar=tqdm.tqdm(range(len(tokenized_POS)))
        pbar.set_description('Positive Validation/Test')
        for i in pbar:
            for token in tokenized_POS[i]:
                pos_prob+=self.positive_word_probs[token]['logprob']
                neg_prob+=self.negative_word_probs[token]['logprob']   
            if pos_prob > neg_prob:                    
                tp+=1
            else:
                fn+=1
        pbar=tqdm.tqdm(range(len(tokenized_NEG)))
        pbar.set_description('Negative Validation/Test')
        pos_prob=self.positive_prob
        neg_prob=self.negative_prob
        for i in pbar:
            for token in tokenized_NEG[i]:
                pos_prob+=self.positive_word_probs[token]['logprob']
                neg_prob+=self.negative_word_probs[token]['logprob']   
            if pos_prob > neg_prob:                    
                fp+=1
            else:
                tn+=1
        acc=(tp+tn)/(tp+tn+fp+fn)
        precision=tp/(tp+fp)
        recall= tp/(tp+fn)
        f1=2*precision*recall/(precision+ recall)
        fpr=fp/(fp+tn)
        fnr=fn/(tp+fn)
        self.metrics={'accuracy': acc, 'precision': precision, 'recall' :recall, 'F1': f1 ,'False Positive Rate':fpr,'False Negative Rate':fnr}
        return tp,tn,fp,fn            
                    

## Seleccion de hyperparametros
Seleccionamos los siguientes Hyperparametros con sus posibles valores

In [ ]:
stopwords=[True,False]
lemmatization=[True,False]
ngrams=[True,False]
n=[2,3,4]
ngrams_mincount=[5,10,15,20]
ngrams_th=[10,15,20,25]

Para poder hacer mas sencilla y rapida la iteracion de estos hyperparametros, generamos los siguientes cuatro sets de datos, combinando los valores de Hyperparametros stopwords y lemmatization, y los guardamos en la carpeta **data/interim** para luego poder cargarlos sin tener que generarlos de nuevo:

Generacion de Datos Lematizados sin stopwords

In [20]:
df = pd.read_csv('data/raw/reviews.csv')
df['rating'] = df['rating'].astype(dtype='int64')
df['sentiment'] = df['rating'].apply(lambda x: Sentiments.POS if x >= 40 else Sentiments.NEG)
review_classes = {sentiment.value: df[df['sentiment'] == sentiment]['review'].values.tolist() for sentiment in Sentiments}
positive_reviews = review_classes['POS']
negative_reviews = review_classes['NEG']
positive_tokens=train_tokenize(True,True,positive_reviews)
negative_tokens=train_tokenize(True,True,negative_reviews)
split_neg=split_data(negative_tokens)
split_pos=split_data(positive_tokens)
with open(r"data/interim/split_pos_lemma_wo_sw.pkl", "wb") as output_file:
    pickle.dump(split_pos, output_file)

with open(r"data/interim/split_neg_lemma_wo_sw.pkl", "wb") as output_file:
    pickle.dump(split_neg, output_file)

Lemmatization:   0%|          | 0/13738 [00:00<?, ?it/s]C:\Users\Ax\anaconda3\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
Remove StopWords: 100%|██████████| 1944/1944 [00:00<00:00, 2592.17it/s]


Generacion de Datos Lematizados con stopwords

In [86]:
df = pd.read_csv('data/raw/reviews.csv')
df['rating'] = df['rating'].astype(dtype='int64')
df['sentiment'] = df['rating'].apply(lambda x: Sentiments.POS if x >= 40 else Sentiments.NEG)
review_classes = {sentiment.value: df[df['sentiment'] == sentiment]['review'].values.tolist() for sentiment in Sentiments}
positive_reviews = review_classes['POS']
negative_reviews = review_classes['NEG']
positive_tokens=train_tokenize(False,True,positive_reviews)
negative_tokens=train_tokenize(False,True,negative_reviews)
split_neg=split_data(negative_tokens)
split_pos=split_data(positive_tokens)

with open(r"data/interim/split_pos_lemma.pkl", "wb") as output_file:
    pickle.dump(split_pos, output_file)

with open(r"data/interim/split_neg_lemma.pkl", "wb") as output_file:
    pickle.dump(split_neg, output_file)

Lemmatization:   0%|          | 0/13738 [00:00<?, ?it/s]C:\Users\Ax\anaconda3\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
Lemmatization: 100%|██████████| 1944/1944 [09:40<00:00,  3.35it/s]


Generacion de Datos no Lematizados con stopwords

In [85]:
df = pd.read_csv('data/raw/reviews.csv')
df['rating'] = df['rating'].astype(dtype='int64')
df['sentiment'] = df['rating'].apply(lambda x: Sentiments.POS if x >= 40 else Sentiments.NEG)
review_classes = {sentiment.value: df[df['sentiment'] == sentiment]['review'].values.tolist() for sentiment in Sentiments}
positive_reviews = review_classes['POS']
negative_reviews = review_classes['NEG']
positive_tokens=train_tokenize(False,False,positive_reviews)
negative_tokens=train_tokenize(False,False,negative_reviews)
split_neg=split_data(negative_tokens)
split_pos=split_data(positive_tokens)

with open(r"data/interim/split_pos.pkl", "wb") as output_file:
    pickle.dump(split_pos, output_file)

with open(r"data/interim/split_neg.pkl", "wb") as output_file:
    pickle.dump(split_neg, output_file)

Sentences to Words: 100%|██████████| 1944/1944 [00:00<00:00, 4238.32it/s]


Generacion de Datos no Lematizados sin stopwords

In [84]:
df = pd.read_csv('data/raw/reviews.csv')
df['rating'] = df['rating'].astype(dtype='int64')
df['sentiment'] = df['rating'].apply(lambda x: Sentiments.POS if x >= 40 else Sentiments.NEG)
review_classes = {sentiment.value: df[df['sentiment'] == sentiment]['review'].values.tolist() for sentiment in Sentiments}
positive_reviews = review_classes['POS']
negative_reviews = review_classes['NEG']
positive_tokens=train_tokenize(True,False,positive_reviews)
negative_tokens=train_tokenize(True,False,negative_reviews)
split_neg=split_data(negative_tokens)
split_pos=split_data(positive_tokens)

with open(r"data/interim/split_pos_wo_sw.pkl", "wb") as output_file:
    pickle.dump(split_pos, output_file)

with open(r"data/interim/split_neg_wo_sw.pkl", "wb") as output_file:
    pickle.dump(split_neg, output_file)

Remove StopWords: 100%|██████████| 1944/1944 [00:00<00:00, 2306.90it/s]


Carga de datasets

In [92]:
positive_docs=[]
negative_docs=[]
with open(r"data/interim/split_pos_lemma_wo_sw.pkl", "rb") as input_file:
    positive_docs.append(pickle.load(input_file))
with open(r"data/interim/split_neg_lemma_wo_sw.pkl", "rb") as input_file:
    negative_docs.append(pickle.load(input_file))
    
with open(r"data/interim/split_pos_lemma.pkl", "rb") as input_file:
    positive_docs.append(pickle.load(input_file))
with open(r"data/interim/split_neg_lemma.pkl", "rb") as input_file:
    negative_docs.append(pickle.load(input_file))
    
with open(r"data/interim/split_pos.pkl", "rb") as input_file:
    positive_docs.append(pickle.load(input_file))
with open(r"data/interim/split_neg.pkl", "rb") as input_file:
    negative_docs.append(pickle.load(input_file))

with open(r"data/interim/split_pos_wo_sw.pkl", "rb") as input_file:
    positive_docs.append(pickle.load(input_file))
with open(r"data/interim/split_neg_wo_sw.pkl", "rb") as input_file:
    negative_docs.append(pickle.load(input_file))

    
hp_key={0:{'lemma':True,'stopwords':True},1:{'lemma':True,'stopwords':False},2:{'lemma':False,'stopwords':False},3:{'lemma':False,'stopwords':True}}

Prueba de Hyperparametros para mejor modelo

In [109]:
j={'ngrams':False,'n':0,'ngrams_mincount':0,'ngrams_th':0}
results=[]

for i in range(len(positive_docs)):
    model=NB_sentiment_analysis(ngrams=False)
    model.fit(positive_docs[i]['train'],negative_docs[i]['train'])
    model.val_test(positive_docs[i]['validation'],negative_docs[i]['validation'])
    results.append(hp_key[i]|j|model.metrics)

n_lst=[2,3,4]
ngrams_mincount_lst=[5,10,15,20]
ngrams_th_lst=[10,15,20,25]
ngrams_i=True
for i in range(len(positive_docs)):
    for n_i in n_lst:
        for ngramas_mincount_i in ngrams_mincount_lst:
            for ngrams_th_i in ngrams_th_lst:
                model=NB_sentiment_analysis(ngrams=True,n=n_i,ngrams_mincount=ngramas_mincount_i,ngrams_th=ngrams_th_i)
                model.fit(positive_docs[i]['train'],negative_docs[i]['train'])
                model.val_test(positive_docs[i]['validation'],negative_docs[i]['validation'])
                results.append(hp_key[i]|{'ngrams':True,'n':n_i,'ngrams_mincount':ngramas_mincount_i,'ngrams_th':ngrams_th_i}|model.metrics)
                 

df_results=pd.DataFrame(results)
print(df_results)
    


2022-07-04 23:39:37.568738
2022-07-04 23:39:37.940450


Negative Validation/Test: 100%|██████████| 309/309 [00:00<00:00, 61893.02it/s]

2022-07-04 23:39:38.030764


2022-07-04 23:39:38.422597


Negative Validation/Test: 100%|██████████| 279/279 [00:00<00:00, 21039.39it/s]

...

Positive class N-grams


Create N-grams: 100%|██████████| 3/3 [00:01<00:00,  2.21it/s]


2022-07-04 23:57:32.135575


Negative Validation/Test: 100%|██████████| 332/332 [00:00<00:00, 65351.46it/s]


     lemma  stopwords  ngrams  n  ngrams_mincount  ngrams_th  accuracy  \
0     True       True   False  0                0          0  0.998740   
1     True      False   False  0                0          0  1.000000   
2    False      False   False  0                0          0  1.000000   
3    False       True   False  0                0          0  0.999567   
4     True       True    True  2                5         10  0.998740   
..     ...        ...     ... ..              ...        ...       ...   
191  False       True    True  4               15         25  0.999567   
192  False       True    True  4               20         10  0.999567   
193  False       True    True  4               20         15  0.999567   
194  False       True    True  4               20         20  0.999567   
195  False       True    True  4               20         25  0.999567   

     precision  recall        F1  False Positive Rate  False Negative Rate  
0     0.998554     1.0  0.999277  

## Discusion de resultados
De los resultados anteriores queremos encontrar el que tenga mejor F1, ya que queremos optimizar tanto los falsos positivos como los falsos negativos

In [110]:
df_results[df_results['F1']==max(df_results['F1'])]

,lemma,stopwords,ngrams,n,ngrams_mincount,ngrams_th,accuracy,precision,recall,F1,False Positive Rate,False Negative Rate
1,True,False,False,0,0,0,1.0,1.0,1.0,1.0,0.0,0.0
2,False,False,False,0,0,0,1.0,1.0,1.0,1.0,0.0,0.0
52,True,False,True,2,5,10,1.0,1.0,1.0,1.0,0.0,0.0
53,True,False,True,2,5,15,1.0,1.0,1.0,1.0,0.0,0.0
54,True,False,True,2,5,20,1.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
143,False,False,True,4,15,25,1.0,1.0,1.0,1.0,0.0,0.0
144,False,False,True,4,20,10,1.0,1.0,1.0,1.0,0.0,0.0
145,False,False,True,4,20,15,1.0,1.0,1.0,1.0,0.0,0.0
146,False,False,True,4,20,20,1.0,1.0,1.0,1.0,0.0,0.0


observamos que 98 de nuestros modelos cuentan con un F1 del 100%, por lo cual escogeremos el cual nos es mas facil y rapido de entrenar y poner en produccion, podemos observar que ese seria el que tiene por hyperparametros, que no se ejecute Lemmatizacion, no se quiten stopwords y no se generen ngrams, unicamente una tokenizacion simple de los documentos, procedemos a probar nuestro mejor modelo:

In [112]:
with open(r"data/interim/split_pos.pkl", "rb") as input_file:
    positive=pickle.load(input_file)
with open(r"data/interim/split_neg.pkl", "rb") as input_file:
    negative=pickle.load(input_file)
    

best_model=NB_sentiment_analysis(stopwords=False,lemmatization=False,ngrams=False)

best_model.fit(positive['train'],negative['train'])
best_model.val_test(positive['validation'],negative['validation'])
print('Validation Metrics')
print(best_model.metrics)
best_model.val_test(positive['test'],negative['test'])
print('Test Metrics')
print(best_model.metrics)

2022-07-05 00:16:32.132284
2022-07-05 00:16:32.861941


Negative Validation/Test: 100%|██████████| 287/287 [00:00<00:00, 12479.17it/s]


Validation Metrics
{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'F1': 1.0, 'False Positive Rate': 0.0, 'False Negative Rate': 0.0}


Negative Validation/Test: 100%|██████████| 307/307 [00:00<00:00, 13348.79it/s]

Test Metrics
{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'F1': 1.0, 'False Positive Rate': 0.0, 'False Negative Rate': 0.0}


## Sanity Check
Primero realizaremos sanity check en el metodo predict, para dos comentarios de diferentes clases

In [117]:
sentence=positive_reviews[10]
print(sentence)
best_model.predict(sentence)
#remove_stopwords(sentences_to_words([sentence]))

Stayed here 3 nights in May. Cute place, very small and quiet. 2 blocks from the central square, but restaurants, shops, churches, etc are all around you. Good value also. Love that breakfast is included.


Sentences to Words: 100%|██████████| 1/1 [00:00<00:00, 999.60it/s]


<Sentiments.POS: 'POS'>

In [118]:
sentence=negative_reviews[10]
print(sentence)
best_model.predict(sentence)
#remove_stopwords(sentences_to_words([sentence]))

Have stayed here a total of nearly 10 weeks over eight visits; my next visit is already reserved. I've been here with family, friends, and strangers about to become friends.  This is not a five-star property.  Its conversion from hacienda to hotel was slap-dash and haphazard. There can, at times, be too much street noise, and the wifi is wonky. ...but I'll be back, and I'll recommend it to friends as an oasis from their expectations.  The courtyards and walk ways will hang with dense flowering vines in the spring and summer, that when added to the flowering shrubs and trees perfumes this small world in a way Chanel aspires to.  The fountain in the main courtyard adds its own bouquet and the soundtrack of splashing water.  It's a perfect place to sip a beer, read a book, fall


Sentences to Words: 100%|██████████| 1/1 [00:00<00:00, 1002.46it/s]


<Sentiments.NEG: 'NEG'>

obtuvimos las salidas deseadas

Realizamos Sanity check de las funciones de tokenizacion:

In [120]:
sentence=negative_reviews[10]
doc=[sentence]
print(doc)
train_tokenize(False,False,doc)


["Have stayed here a total of nearly 10 weeks over eight visits; my next visit is already reserved. I've been here with family, friends, and strangers about to become friends.  This is not a five-star property.  Its conversion from hacienda to hotel was slap-dash and haphazard. There can, at times, be too much street noise, and the wifi is wonky. ...but I'll be back, and I'll recommend it to friends as an oasis from their expectations.  The courtyards and walk ways will hang with dense flowering vines in the spring and summer, that when added to the flowering shrubs and trees perfumes this small world in a way Chanel aspires to.  The fountain in the main courtyard adds its own bouquet and the soundtrack of splashing water.  It's a perfect place to sip a beer, read a book, fall"]


Sentences to Words: 100%|██████████| 1/1 [00:00<00:00, 999.60it/s]


[['have',
  'stayed',
  'here',
  'total',
  'of',
  'nearly',
  'weeks',
  'over',
  'eight',
  'visits',
  'my',
  'next',
  'visit',
  'is',
  'already',
  'reserved',
  've',
  'been',
  'here',
  'with',
  'family',
  'friends',
  'and',
  'strangers',
  'about',
  'to',
  'become',
  'friends',
  'this',
  'is',
  'not',
  'five',
  'star',
  'property',
  'its',
  'conversion',
  'from',
  'hacienda',
  'to',
  'hotel',
  'was',
  'slap',
  'dash',
  'and',
  'haphazard',
  'there',
  'can',
  'at',
  'times',
  'be',
  'too',
  'much',
  'street',
  'noise',
  'and',
  'the',
  'wifi',
  'is',
  'wonky',
  'but',
  'll',
  'be',
  'back',
  'and',
  'll',
  'recommend',
  'it',
  'to',
  'friends',
  'as',
  'an',
  'oasis',
  'from',
  'their',
  'expectations',
  'the',
  'courtyards',
  'and',
  'walk',
  'ways',
  'will',
  'hang',
  'with',
  'dense',
  'flowering',
  'vines',
  'in',
  'the',
  'spring',
  'and',
  'summer',
  'that',
  'when',
  'added',
  'to',
  'the',

In [121]:
sentence=negative_reviews[10]
doc=[sentence]
print(doc)
train_tokenize(False,True,doc)

["Have stayed here a total of nearly 10 weeks over eight visits; my next visit is already reserved. I've been here with family, friends, and strangers about to become friends.  This is not a five-star property.  Its conversion from hacienda to hotel was slap-dash and haphazard. There can, at times, be too much street noise, and the wifi is wonky. ...but I'll be back, and I'll recommend it to friends as an oasis from their expectations.  The courtyards and walk ways will hang with dense flowering vines in the spring and summer, that when added to the flowering shrubs and trees perfumes this small world in a way Chanel aspires to.  The fountain in the main courtyard adds its own bouquet and the soundtrack of splashing water.  It's a perfect place to sip a beer, read a book, fall"]


Lemmatization:   0%|          | 0/1 [00:00<?, ?it/s]C:\Users\Ax\anaconda3\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
Lemmatization: 100%|██████████| 1/1 [00:00<00:00,  1.79it/s]


[['stay',
  'here',
  'total',
  'nearly',
  'week',
  'visit',
  'next',
  'visit',
  'already',
  'reserved',
  've',
  'here',
  'family',
  'friend',
  'stranger',
  'about',
  'become',
  'friend',
  'star',
  'property',
  'conversion',
  'hacienda',
  'hotel',
  'slap',
  'dash',
  'haphazard',
  'time',
  'too',
  'much',
  'street',
  'noise',
  'wifi',
  'wonky',
  'back',
  'recommend',
  'friend',
  'oasis',
  'expectation',
  'courtyard',
  'walk',
  'way',
  'hang',
  'dense',
  'flower',
  'vine',
  'spring',
  'summer',
  'add',
  'flower',
  'shrub',
  'tree',
  'perfume',
  'small',
  'world',
  'way',
  'aspire',
  'fountain',
  'main',
  'courtyard',
  'add',
  'own',
  'bouquet',
  'soundtrack',
  'splash',
  'water',
  'perfect',
  'place',
  'sip',
  'beer',
  'read',
  'book']]

In [122]:
sentence=negative_reviews[10]
doc=[sentence]
print(doc)
train_tokenize(True,True,doc)

["Have stayed here a total of nearly 10 weeks over eight visits; my next visit is already reserved. I've been here with family, friends, and strangers about to become friends.  This is not a five-star property.  Its conversion from hacienda to hotel was slap-dash and haphazard. There can, at times, be too much street noise, and the wifi is wonky. ...but I'll be back, and I'll recommend it to friends as an oasis from their expectations.  The courtyards and walk ways will hang with dense flowering vines in the spring and summer, that when added to the flowering shrubs and trees perfumes this small world in a way Chanel aspires to.  The fountain in the main courtyard adds its own bouquet and the soundtrack of splashing water.  It's a perfect place to sip a beer, read a book, fall"]


Remove StopWords: 100%|██████████| 1/1 [00:00<00:00, 63.13it/s]


[['stay',
  'total',
  'nearly',
  'week',
  'visit',
  'next',
  'visit',
  'already',
  'reserved',
  'family',
  'friend',
  'stranger',
  'become',
  'friend',
  'star',
  'property',
  'conversion',
  'hacienda',
  'hotel',
  'slap',
  'dash',
  'haphazard',
  'time',
  'much',
  'street',
  'noise',
  'wifi',
  'wonky',
  'back',
  'recommend',
  'friend',
  'oasis',
  'expectation',
  'courtyard',
  'walk',
  'way',
  'hang',
  'dense',
  'flower',
  'vine',
  'spring',
  'summer',
  'add',
  'flower',
  'shrub',
  'tree',
  'perfume',
  'small',
  'world',
  'way',
  'aspire',
  'fountain',
  'main',
  'courtyard',
  'add',
  'bouquet',
  'soundtrack',
  'splash',
  'water',
  'perfect',
  'place',
  'sip',
  'beer',
  'read',
  'book']]